<a href="https://colab.research.google.com/github/kunamneniSravya/GEN-AI/blob/main/Bert_predicts_mlm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import required libraries
from transformers import AutoTokenizer, AutoModelForMaskedLM
from scipy.special import softmax
import numpy as np

In [2]:
# Specify the pre-trained model to use: BERT-base-cased
model_name = "bert-base-cased"

# Instantiate the tokenizer and model for the specified pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [28]:
# Get the mask token from the tokenizer
mask = tokenizer.mask_token
print(mask)

[MASK]


In [17]:
# Create a sentence with a mask token to be filled in by the model
sentence = f"I want to {mask} home for preparation holidays."
# Tokenize the sentence
tokens = tokenizer.tokenize(sentence)
print(tokens)

['I', 'want', 'to', '[MASK]', 'home', 'for', 'preparation', 'holidays', '.']


In [18]:
# Encode the sentence using the tokenizer and return the input tensors
encoded_inputs = tokenizer(sentence, return_tensors='pt')
print(encoded_inputs)

{'input_ids': tensor([[  101,   146,  1328,  1106,   103,  1313,  1111,  7288, 13413,   119,
           102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [19]:
# Get the model's output for the input tensors
outputs = model(**encoded_inputs)
print(outputs)

MaskedLMOutput(loss=None, logits=tensor([[[ -7.2444,  -7.1491,  -7.3336,  ...,  -6.1504,  -5.7592,  -6.2374],
         [ -7.9007,  -8.1815,  -7.9517,  ...,  -6.6035,  -6.3348,  -6.8491],
         [-10.5887, -10.3088, -10.4009,  ...,  -6.5393,  -5.9455,  -6.3158],
         ...,
         [ -6.8176,  -6.6027,  -6.4963,  ...,  -6.6034,  -5.2091,  -5.8339],
         [-12.4532, -12.3151, -12.4035,  ...,  -9.6021,  -9.5701, -10.1945],
         [-12.3164, -12.2773, -12.6777,  ..., -10.0134, -10.2330, -11.0119]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)


In [20]:
# Detach the logits from the model's output and convert them to numpy arrays
logits = outputs.logits.detach().numpy()[0]
logits.shape

(11, 28996)

In [21]:
len(tokens)

9

In [22]:
# Extract the logits for the mask token
mask_logits = logits[tokens.index(mask) + 1]
print(mask_logits)

[-5.7572317 -5.58551   -5.4140277 ... -3.8158064 -3.6005135 -2.759586 ]


In [26]:
mask_logits.shape

(28996,)

In [23]:
# Calculate the confidence scores for each possible token using softmax
confidence_scores = softmax(mask_logits)
print(confidence_scores)

[5.5138777e-10 6.5468930e-10 7.7715734e-10 ... 3.8424433e-09 4.7654987e-09
 1.1048883e-08]


In [24]:
confidence_scores.sum()

0.9999997

In [27]:
# Print the top 5 predicted tokens and their confidence scores
for i in np.argsort(confidence_scores)[::-1][:10]:
    pred_token = tokenizer.decode(i)
    score = confidence_scores[i]

    # Print the predicted sentence with the mask token replaced by the predicted token, and the confidence score
    print(sentence.replace(mask, pred_token), score)

I want to go home for preparation holidays. 0.48090363
I want to be home for preparation holidays. 0.18020684
I want to come home for preparation holidays. 0.16811372
I want to stay home for preparation holidays. 0.051831406
I want to get home for preparation holidays. 0.03344697
I want to return home for preparation holidays. 0.019068364
I want to head home for preparation holidays. 0.018434305
I want to travel home for preparation holidays. 0.0071388152
I want to fly home for preparation holidays. 0.006187038
I want to leave home for preparation holidays. 0.0039153565
